In [1]:
import os
import random 
import pandas as pd
import warnings
import sys
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from tqdm import tqdm

# local imports
sys.path.append(r"./utils")
from utils import utils

random.seed(1234)
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
COLAB = False
DEBUG = True
DATA_DIR = "./data/full/"
results_file_loc = "results/Model results/segment_results.csv"

if DEBUG:
    DATA_DIR = "./data/sample/"

DATA_DROP_COLS = \
           ['Unnamed: 0', # index columns
           
            'gender', # self reported and filtered already
            'country', # self reported and filtered already
            'userid', # index equivalent column 
            'pol_dat_us', # redundant columns with label
            'pol_dat_ca', # redundant columns with label
            'pol_dat_uk', # redundant columns with label
            'pol_fb_us', # redundant columns with label
            'database', # filtered already 
            # 'ethnicity.value' # filtered already
            ]

In [3]:
if COLAB:
  from google.colab import drive
  drive.mount('/content/drive')
    
  #TODO: untested for COLAB == True, more might be needed here
  DATA_DIR = "/content/drive/Shareddrives/Facial Recognition/data/"

In [4]:
# List of data file paths

folders = os.listdir(DATA_DIR)
dataset_paths = []
results = []
problematic_data = []
for folder in tqdm(folders):
  
  if folder != "NO FILES":
    # which group it is being processed on 
    group_name = folder
    
    only_image_cols = list(map(str, range(1,2049))) + ['pol']
    image_self_reported = only_image_cols + ['age']
    data = utils.get_segment_dataframe(segment_to_run = folder)

    data = data.drop(DATA_DROP_COLS,axis=1)

    data = utils.get_clean_data(data)

    try:
      dataframe_dict = {
      "Only Image Features" : data[only_image_cols],
      "Image and Self Reported Features" : data[image_self_reported],
      "Image and extracted Features":data.drop("age",axis = 1),
      "Image, Self-reported and Extracted Features": data
    }
    
      for data_name, data_set in dataframe_dict.items():
        for model_name in ["NN","LR"]:
          auc, acc = utils.fit_and_get_metrics(data_set, model_name)
          results.append([group_name,model_name,data_name,auc,acc])
    except:
      problematic_data.append([group_name]) # DEBUG to check if the dataframe is off somewhere. 
      
  # break

utils.save_results(results_array=results, location=results_file_loc)

if problematic_data:
  print(problematic_data)
    

100%|██████████| 9/9 [10:40<00:00, 71.19s/it]

 Segment Results Saved !!
[]


hell
